# **Data Science Projekt - Datenvorbereitung**

## **Überblick**
Dieses Notebook führt die Datenvorbereitung für unser Data Science Projekt durch. Es umfasst:

1. **Datenimport und -vereinheitlichung** von News-CSV-Dateien
2. **Zeitstempel-Verarbeitung** und Kategorisierung
3. **Datenaggregation** für verschiedene Metriken
4. **Zusammenführung** mit Chart-Daten
5. **Export** der finalen Datei

## **Datenquellen**
- **News-Daten**: CSV-Dateien aus `data/raw/news/` (verschiedene Kategorien)
- **Chart-Daten**: `data/raw/chart/raw_chart_data.csv`

---

##  **1. Setup und Imports**

Laden der erforderlichen Bibliotheken und Definition von Konstanten.

In [ ]:
# standard libs
from typing import Dict

# data science libs
import pandas as pd
from IPython.display import display

# import utility functions
from utils.data_prep_utils import (
    load_and_process_news_data,
    aggregate_news_data,
    analyze_column_quality,
    process_chart_data,
    merge_and_finalize_data,
    print_final_summary
)

# configuration
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# constants
CSV_FOLDER = "data/raw/news/"
CHART_DATA_PATH = "data/raw/chart/raw_chart_data.csv"
OUTPUT_PATH = "data/merged/merged_by_timestamp.csv"

# Mapping for impact levels
IMPACT_MAPPING: Dict[str, int] = {
    "NONE": 0,
    "LOW": 1, 
    "MEDIUM": 2,
    "HIGH": 3
}

print("Setup completed")
print(f"News data folder: {CSV_FOLDER}")
print(f"Chart data path: {CHART_DATA_PATH}")
print(f"Output path: {OUTPUT_PATH}")

## **2. Daten einlesen**

### Ziel
Laden aller News-CSV-Dateien aus verschiedenen Kategorien und Vereinheitlichung der Datenstruktur.

### Prozess
1. **Dateien scannen**: Alle CSV-Dateien im News-Ordner finden
2. **Zeitstempel konvertieren**: String-Format in DateTime umwandeln
3. **Impact-Mapping**: Kategorische Impact-Werte in numerische Werte umwandeln
4. **Kategorisierung**: Dateiname als Kategorie hinzufügen
5. **Zusammenführung**: Alle DataFrames zu einem Master-DataFrame kombinieren
6. **Sortierung**: Chronologische Anordnung nach Zeitstempel

### Impact-Mapping
- `NONE` → 0 (Kein Einfluss)
- `LOW` → 1 (Geringer Einfluss)  
- `MEDIUM` → 2 (Mittlerer Einfluss)
- `HIGH` → 3 (Hoher Einfluss)

In [ ]:
# Load and process news data using utility function
final_df, stats_df = load_and_process_news_data(
    csv_folder=CSV_FOLDER,
    impact_mapping=IMPACT_MAPPING,
    verbose=True
)

# Display processing statistics
print("\nProcessing statistics:")
display(stats_df)

print("\nFirst 5 rows of the master DataFrame:")
display(final_df.head())

## **3. Zeitstempel-Rundung und Datenaggregation**

### Warum Zeitstempel-Rundung?
Da die News-Events zu verschiedenen exakten Zeiten stattfinden, runden wir alle Timestamps auf die **nächste halbe Stunde** auf. Dies ermöglicht:
- **Konsistente Zeitintervalle** für die Analyse
- **Gruppierung** mehrerer Events im gleichen Zeitfenster
- **Bessere Korrelation** mit Chart-Daten (die ebenfalls in 30-Minuten-Intervallen vorliegen)

### Rundungslogik
- **Exakt 00:00 oder 30:00**: Keine Änderung
- **00:01 - 29:59**: Aufrunden auf 30:00
- **30:01 - 59:59**: Aufrunden auf nächste volle Stunde

### Aggregierte Metriken

#### Basis-Metriken (pro Zeitstempel)
- **Event Count**: Anzahl aller News-Events
- **Impact Statistiken**: Summe, Mittelwert, Maximum, Minimum, Standardabweichung
- **Impact Verteilung**: Anzahl Events pro Impact-Level (0-3)
- **Impact Diversität**: Anzahl unterschiedlicher Impact-Level

#### Kategorie-spezifische Metriken
Für jede News-Kategorie separat:
- **Event Count**: Anzahl Events pro Kategorie
- **Impact Sum**: Gesamter Impact pro Kategorie  
- **Impact Max**: Höchster Impact pro Kategorie

In [ ]:
# Aggregate news data using utility function
agg = aggregate_news_data(final_df, verbose=True)

# Overview of created features
print("\nÜbersicht der erstellten Features:")
feature_groups = {
    'Basis-Metriken': [col for col in agg.columns if not col.startswith('cat_') and col != 'Timestamp'],
    'Kategorie-Metriken': [col for col in agg.columns if col.startswith('cat_')]
}

for group, features in feature_groups.items():
    print(f"\n{group} ({len(features)} Features):")
    for feature in features:
        print(f"  • {feature}")

print("\nErste 5 Zeilen der aggregierten Daten:")
display(agg.head())

## **4. Datenqualitätsprüfung**

### Überprüfung auf Null-Spalten
Identifikation von Spalten, die ausschließlich den Wert 0 enthalten. Dies hilft bei:

- **Data Cleaning**: Entfernung redundanter Features
- **Feature Selection**: Fokus auf informative Variablen  
- **Speicheroptimierung**: Reduzierung der Dateigröße
- **Model Performance**: Vermeidung uninformativer Features

### Warum entstehen Null-Spalten?
- **Seltene Kategorien**: Manche News-Kategorien haben möglicherweise keine Events in bestimmten Zeiträumen
- **Aggregation**: Durch die Zeitstempel-Rundung können manche Kombinationen leer werden
- **Datenqualität**: Fehlende oder unvollständige Quelldaten

In [ ]:
# Analyze column quality using utility function
only_zero_cols, column_stats = analyze_column_quality(agg, verbose=True)

## **5. Integration mit Chart-Daten und Finalisierung**

### Ziel der Integration
Zusammenführung der aggregierten News-Metriken mit den Finanzchart-Daten zur Erstellung eines **Master-Datasets** für Machine Learning Analysen.

### Chart-Daten Verarbeitung
1. **OHLC-Daten**: Open, High, Low, Close Preise
2. **Return-Berechnung**: Prozentuale Preisveränderung basierend auf größter Bewegung
3. **Volumen**: Handelsvolumen hinzufügen

### Return-Berechnung Logik
```
Return = (Größte_Bewegung - Open) / Open * 100

Größte_Bewegung = {
    Low,   falls |Open - Low| > |High - Open|
    High,  sonst
}
```

### Zeitzone-Anpassung
- **Problem**: Chart-Daten in UTC, US-Märkte in Eastern Time
- **Lösung**: Automatische Daylight Saving Time (DST) Erkennung und Anpassung
- **Ergebnis**: Synchronisierte Zeitstempel für korrekte Korrelation

### Data Merge Strategie
- **Join-Typ**: Left Join (Chart-Daten als Basis)
- **Join-Key**: Timestamp (nach Rundung und Zeitzone-Anpassung)
- **Missing Values**: Mit 0 auffüllen (= keine News-Aktivität)

### Finale Filterung
- **Wochenenden entfernen**: Sonntags-Daten ausschließen (Markt geschlossen)
- **Nur Handelszeiten**: Fokus auf relevante Zeitperioden

In [ ]:
# Step 4: Process chart data and merge with news data
print("Starte finale Datenverarbeitung...")

# Load and process chart data
df_chart = process_chart_data(CHART_DATA_PATH, verbose=True)

# Merge all data and apply final processing
df_merged = merge_and_finalize_data(
    df_chart=df_chart,
    df_news=agg,
    output_path=OUTPUT_PATH,
    verbose=True
)

# Print comprehensive summary
print_final_summary(df_merged, verbose=True)

print("\nErste 5 Zeilen des finalen Datasets:")
display(df_merged.head())

## **6. Datenvorbereitung Abgeschlossen**

### Was wurde erreicht?

Das Notebook hat erfolgreich ein **Machine Learning-ready Dataset** erstellt durch:

1. **Datenintegration**: 6 News-Kategorien + Chart-Daten zusammengeführt
2. **Zeitharmonisierung**: Timestamps auf 30-Minuten-Intervalle standardisiert  
3. **Feature Engineering**: 40+ quantitative Features aus News-Events generiert
4. **Zeitzone-Korrektheit**: UTC/Eastern Time Synchronisation mit DST-Behandlung
5. **Datenbereinigung**: Wochenend-Filterung und Qualitätskontrolle

### Dataset-Übersicht

| Aspekt | Details |
|--------|---------|
| **Format** | CSV-Datei, ML-ready |
| **Größe** | ~2MB, optimiert für Analyse |
| **Zeitraum** | Mehrere Monate Finanzdaten |
| **Frequenz** | 30-Minuten Intervalle |
| **Features** | 6 Chart + 40+ News Features |
| **Target** | `Return` (prozentuale Preisveränderung) |


### Output
**Hauptdatei**: `data/merged/merged_by_timestamp.csv`